<a href="https://colab.research.google.com/github/Aradhyakapil/CodeGen-Agent-LangGraph/blob/main/CodeGen_Agent_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai langchain langchain-openai langgraph requests

In [ ]:
from langchain_openai import OpenAI as LC_OpenAI
from langchain import PromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict
import os
import time
import requests
from google.colab import userdata

# ─────── Environment Setup ───────
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
JUDGE0_API_KEY = userdata.get("JUDGE0_API_KEY").strip()

# ─────── Shared State Type ───────
class GraphState(TypedDict):
    messages: list
    language: str
    user_request: str
    code: str
    previous_code: str
    result: dict

# ─────── Node 1: Code Generator with Intent-Based Memory ───────
prompt_template_with_memory = PromptTemplate(
    input_variables=["language", "previous_code", "user_request"],
    template="""
You are an AI assistant generating code using GPT‑4o‑mini.

The user wants code in {language}.
Here is the previous code to build upon:
{previous_code}

User request:
{user_request}

Modify or extend the previous code as per the user request.
Provide a complete code snippet without explanation.
STRICTLY Do not include backticks or ```python.
STRICTLY Do not include explanations.
"""
)

prompt_template_fresh = PromptTemplate(
    input_variables=["language", "user_request"],
    template="""
You are an AI assistant generating code using GPT‑4o‑mini.

The user wants code in {language}.

User request:
{user_request}

Provide a complete code snippet without explanation.
STRICTLY Do not include backticks or ```python.
STRICTLY Do not include explanations.
"""
)

llm = LC_OpenAI(model_name="gpt-4o-mini", temperature=0.5)

def generate_code_node(state: GraphState) -> GraphState:
    lang_input = state["language"] or "python"
    user_req = state["user_request"]
    prev_code = state["previous_code"] or ""

    if "previous code" in user_req.lower() and prev_code:
        # User explicitly asked to use previous code
        chain = prompt_template_with_memory | llm
        code = chain.invoke({
            "language": lang_input,
            "previous_code": prev_code,
            "user_request": user_req
        })
    else:
        # Fresh request
        chain = prompt_template_fresh | llm
        code = chain.invoke({
            "language": lang_input,
            "user_request": user_req
        })

    state["code"] = code
    return state

# ─────── Node 2: Judge0 Code Runner ───────
def run_code_node(state: GraphState) -> GraphState:
    language_ids = {"python": 71, "javascript": 63, "java": 62, "c++": 54, "c": 50}
    lang = state["language"].lower() or "python"
    code = state["code"]
    if lang not in language_ids:
        state["result"] = {"error": f"Unsupported language: {lang}"}
        return state

    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": JUDGE0_API_KEY,
        "X-RapidAPI-Host": "judge0-ce.p.rapidapi.com"
    }
    payload = {
        "source_code": code,
        "language_id": language_ids[lang],
        "cpu_time_limit": 2,
        "memory_limit": 128000,
        "encode_base64": False,
        "wait": False
    }
    try:
        response = requests.post("https://judge0-ce.p.rapidapi.com/submissions", headers=headers, json=payload)
        response.raise_for_status()
        token = response.json().get("token")
        if not token:
            state["result"] = {"error": "Failed to retrieve submission token."}
            return state
        status_url = f"https://judge0-ce.p.rapidapi.com/submissions/{token}"
        for _ in range(10):
            result = requests.get(status_url, headers=headers).json()
            if result.get("status", {}).get("id", 0) in [1, 2]:
                time.sleep(1)
                continue
            state["result"] = {
                "status": result.get("status", {}).get("description"),
                "stdout": result.get("stdout", ""),
                "stderr": result.get("stderr", ""),
                "compile_output": result.get("compile_output", ""),
                "message": result.get("message", "")
            }
            return state
        state["result"] = {"error": "Timed out waiting for result."}
        return state
    except requests.RequestException as e:
        state["result"] = {"error": f"Network or Judge0 API error: {e}"}
        return state

# ─────── Node 3: Assistant Reply ───────
def reply_node(state: GraphState) -> GraphState:
    content = (
        f"Generated Code:\n{state['code']}\n\n"
        f"Execution Result:\n{state['result']}"
    )
    state['messages'].append({"role": "assistant", "content": content})
    state['previous_code'] = state['code']  # Save for future reference
    return state

# ─────── LangGraph Construction with Memory ───────
memory = MemorySaver()
graph_builder = StateGraph(GraphState)

graph_builder.add_node("generate_code", generate_code_node)
graph_builder.add_node("run_code", run_code_node)
graph_builder.add_node("reply", reply_node)

graph_builder.set_entry_point("generate_code")
graph_builder.add_edge("generate_code", "run_code")
graph_builder.add_edge("run_code", "reply")
graph_builder.add_edge("reply", END)

graph = graph_builder.compile(checkpointer=memory)

# ─────── Interactive Chat Loop ───────
def chat():
    thread_id = input("Enter conversation thread ID: ").strip() or "1"
    config = {"configurable": {"thread_id": thread_id}}
    print("Start chatting! Type 'exit' to quit.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            break
        language = input("Language (press Enter for Python): ").strip().lower() or "python"

        snapshot = graph.get_state(config)
        msgs = snapshot.values.get('messages', []) if snapshot else []
        previous_code = snapshot.values.get('previous_code', "") if snapshot else ""
        previous_result = snapshot.values.get('result', {}) if snapshot else {}

        print("\n[Previous Code Available]" if previous_code else "\n[No Previous Code]")
        print("[Previous Execution Result Available]" if previous_result else "[No Previous Result]")

        msgs.append({"role": "user", "content": user_input})
        state = {
            "messages": msgs,
            "language": language,
            "user_request": user_input,
            "code": "",
            "previous_code": previous_code,
            "result": {}
        }
        for _ in graph.stream(state, config, stream_mode="values"):
            pass
        new_snapshot = graph.get_state(config)
        last_msg = new_snapshot.values['messages'][-1]['content']
        print(f"Assistant: {last_msg}\n")

if __name__ == "__main__":
    chat()


Enter conversation thread ID: 14
Start chatting! Type 'exit' to quit.
You: sum first 100 natural numbers
Language (press Enter for Python): python

[No Previous Code]
[No Previous Result]
Assistant: Generated Code:
sum_natural_numbers = sum(range(1, 101))
print(sum_natural_numbers)

Execution Result:
{'status': 'Accepted', 'stdout': '5050\n', 'stderr': None, 'compile_output': None, 'message': None}

You: Modify previous code to also print the average
Language (press Enter for Python): python

[Previous Code Available]
[Previous Execution Result Available]
Assistant: Generated Code:
sum_natural_numbers = sum(range(1, 101))
average_natural_numbers = sum_natural_numbers / 100
print(sum_natural_numbers)
print(average_natural_numbers)

Execution Result:
{'status': 'Accepted', 'stdout': '5050\n50.5\n', 'stderr': None, 'compile_output': None, 'message': None}

You: exit
